# ベイズ推論による機械学習入門
## 3.5 線型回帰

### Example

In [ ]:
using LinearAlgebra
using Distributions
using Plots
gr()

In [ ]:
M = 4 # dimensions
N = 20 # sampling data
xmin = 0
xmax = 5
xs = range(xmin, xmax, length=100)
x_sample = xmin .+ (xmax-xmin) .* rand(N)

lambda = 10 # accuracy parameter
dist_epsilon = Normal(0, lambda^(-1))
epsilon = rand(dist_epsilon, N) # noise epsilon

# parameters
m = zeros(M)
l_lambda = Matrix{Float64}(I, M, M)

In [ ]:
function polynomial(x)
    return [1.0, x, x^2, x^3]
end

function true_func(x, gain, omega)
    return gain * sin(x * omega)
end

function sampling_y(x, gain, omega, var=1.0)
    dist_y = Normal(true_func(x, gain, omega), var)
    return rand(dist_y, 1)[1]
end

In [ ]:
# smapling weight
l_lambda_inv = inv(l_lambda)
ws = rand(MultivariateNormal(m, diag(l_lambda_inv)), 5)'

f(i) = [dot(ws[i, :], polynomial(x)) for x in range(-1, 1, length=1000)]
g() = [f(i) for i in 1:5]

sample_prior = g()
plot(range(-1, 1, length=1000), sample_prior)

In [ ]:
gain = 1
omega = 0.5

# true distribution
f() = [true_func(x, gain, omega) for x in xs]
y_line = f()

# measured data
f() = [sampling_y(x, gain, omega, 1.0/lambda)+epsilon[i] for (i, x) in enumerate(x_sample)]
y_true = f()

plot(x_sample, y_true, seriestype=:scatter, label="measured")
plot!(xs, y_line, label="true")

----
### 事後分布

$$ p(\mathbf{w}|\mathbf{Y},\mathbf{X})=N(\mathbf{w}|\mathbf{\hat{m}},\mathbf{\hat{\Lambda}}^{-1}) $$
$$ \mathbf{\hat{\Lambda}} = \lambda \sum^{N}_{n=1} \mathbf{x}_{n}\mathbf{x}^{\top}_{n} + \mathbf{\Lambda} $$
$$ \mathbf{\hat{m}} = \mathbf{\hat{\Lambda}}^{-1} \lambda (\sum^{N}_{n=1} y_{n}\mathbf{x}_{n} + \mathbf{\Lambda}\mathbf{m}) $$

In [ ]:
println("lambda:", lambda)
l_lambda

In [ ]:
# calculation large lambda hat
f() = [polynomial(x)*polynomial(x)' for x in x_sample]
sum_x = sum(f())

l_lambda_hat = lambda * sum_x .+ l_lambda
l_lambda_hat_inv = inv(l_lambda_hat)
println("Large lambda hat")
l_lambda_hat

In [ ]:
# calculation m_hat
f() = [polynomial(x)*y_true[i] for (i, x) in enumerate(x_sample)]
sum_xy = sum(f())
println("sum_xy:", sum_xy)

m_hat = l_lambda_hat_inv * lambda * sum_xy + l_lambda * m
print("m_hat:", m_hat)

In [ ]:
# calc weight
ws_pred = rand(MultivariateNormal(m_hat, diag(l_lambda_hat_inv)), 5)'

In [ ]:
# plot prediction
f(i) = [dot(ws_pred[i, :], polynomial(x)) for x in xs]
g() = [f(i) for i in 1:5]

sample_prior = g()
plot!(xs, sample_prior, label="predict")

----
### 予測分布

$$
p(y_{*}|\mathbf{x_{*}}) = N(y_{*}|\mu_{*},\lambda_{*}^{-1})\\
\mu_{*} = \hat{\mathbf{m}}^{\top}\mathbf{x}_{*} \\
\lambda_{*}^{-1} = \lambda^{-1}+\mathbf{x}_{*}^{\top}\hat{\mathbf{\Lambda}}^{-1}\mathbf{x}_{*}
$$

In [ ]:
# μ*
f() = [m_hat' * polynomial(x) for x in xs]
mu_star = f()

# lambda*
function lambda_pred(x, l_lambda_hat_inv, lambda)
    return lambda^(-1) + polynomial(x)' * l_lambda_hat_inv * polynomial(x)
end
f() = [lambda_pred(x, l_lambda_hat_inv, lambda) for x in xs]
var_star = f()
print()

In [ ]:
plot(x_sample, y_true, seriestype=:scatter, label="measured")
plot!(xs, y_line, label="true")
plot!(xs, sample_prior, label="predict", color="black")

# mean predicted function
plot!(xs, mu_star, label="mean", color="green")

In [ ]:
# plotting forecast variance
plot(xs, var_star, label="forecast variance")